In [1]:
import os
new_directory = "C:/Users/shaur/Downloads"
os.chdir(new_directory)

print("Current working directory:", os.getcwd())

Current working directory: C:\Users\shaur\Downloads


In [2]:
!curl -L https://raw.githubusercontent.com/Denis2054/Transformers-for-NLP-2nd-Edition/master/Chapter04/kant.txt --output "kant.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 54 10.7M   54 6038k    0     0  5079k      0  0:00:02  0:00:01  0:00:01 5078k
100 10.7M  100 10.7M    0     0  6498k      0  0:00:01  0:00:01 --:--:-- 6497k


In [3]:
%%time
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
paths=[str(x) for x in Path(".").glob("**/*.txt")]

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>","<pad>","</s>","<unk>","<mask>",
])

CPU times: total: 4.39 s
Wall time: 1.74 s


In [4]:
import os
token_dir = 'C:/Users/shaur/Downloads/KantaiBERT'
if not os.path.exists(token_dir):
  os.makedirs(token_dir)
tokenizer.save_model('KantaiBERT')

['KantaiBERT\\vocab.json', 'KantaiBERT\\merges.txt']

In [5]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
tokenizer = ByteLevelBPETokenizer(
"./KantaiBERT/vocab.json",
"./KantaiBERT/merges.txt",
)

In [6]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.']

In [7]:
tokenizer.encode("The Critique of Pure Reason.")

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [8]:
tokenizer._tokenizer.post_processor = BertProcessing(
("</s>", tokenizer.token_to_id("</s>")),
("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [9]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['<s>', 'The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.', '</s>']

In [10]:
!nvidia-smi

Fri Aug 11 22:11:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P0              23W /  67W |      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
import torch
torch.cuda.is_available()

True

In [12]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

c:\Users\shaur\anaconda3\envs\denis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

In [14]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("./KantaiBERT", max_length=512)

In [15]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print(model)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [16]:
print(model.num_parameters())

83504416


In [17]:
LP=list(model.parameters())
lp=len(LP)
print(lp)

106


In [18]:
np=0
for p in range(0,lp):
    PL2=True
    try:
        L2=len(LP[p][0]) #check if 2D
    except:
        L2=1 #not 2D but 1D
        PL2=False
    L1 = len(LP[p])
    L3 = L1*L2
    np+=L3
print(np)

83504416


In [19]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./kant.txt",
    block_size=128,
)

c:\Users\shaur\anaconda3\envs\denis\lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: total: 3.86 s
Wall time: 13.9 s


In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [21]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
output_dir="./KantaiBERT",
overwrite_output_dir=True,
num_train_epochs=1,
per_device_train_batch_size=64,
save_steps=10_000,
save_total_limit=2,
)
trainer = Trainer(
model=model,
args=training_args,
data_collator=data_collator,
train_dataset=dataset,
)

In [22]:
%%time
trainer.train()

c:\Users\shaur\anaconda3\envs\denis\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 19%|█▉        | 501/2672 [01:26<05:33,  6.50it/s]

{'loss': 6.6053, 'learning_rate': 4.06437125748503e-05, 'epoch': 0.19}


 37%|███▋      | 1001/2672 [02:49<04:46,  5.84it/s]

{'loss': 5.7151, 'learning_rate': 3.12874251497006e-05, 'epoch': 0.37}


 56%|█████▌    | 1501/2672 [04:12<03:17,  5.91it/s]

...

 94%|█████████▎| 2501/2672 [06:59<00:28,  6.02it/s]

{'loss': 4.8477, 'learning_rate': 3.218562874251497e-06, 'epoch': 0.94}


100%|██████████| 2672/2672 [07:28<00:00,  5.96it/s]

{'train_runtime': 448.0592, 'train_samples_per_second': 381.566, 'train_steps_per_second': 5.963, 'train_loss': 5.4406075391940725, 'epoch': 1.0}
CPU times: total: 1min 52s
Wall time: 7min 28s


TrainOutput(global_step=2672, training_loss=5.4406075391940725, metrics={'train_runtime': 448.0592, 'train_samples_per_second': 381.566, 'train_steps_per_second': 5.963, 'train_loss': 5.4406075391940725, 'epoch': 1.0})

In [23]:
trainer.save_model("./KantaiBERT")

In [24]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer="./KantaiBERT"
)

In [25]:
fill_mask("Human thinking involves human <mask>.")

[{'score': 0.038521718233823776,
  'token': 404,
  'token_str': ' reason',
  'sequence': 'Human thinking involves human reason.'},
 {'score': 0.016436902806162834,
  'token': 407,
  'token_str': ' object',
  'sequence': 'Human thinking involves human object.'},
 {'score': 0.01360483467578888,
  'token': 621,
  'token_str': ' intuition',
  'sequence': 'Human thinking involves human intuition.'},
 {'score': 0.010682177729904652,
  'token': 561,
  'token_str': ' experience',
  'sequence': 'Human thinking involves human experience.'},
 {'score': 0.01002083532512188,
  'token': 456,
  'token_str': ' law',
  'sequence': 'Human thinking involves human law.'}]

In [30]:
fill_mask("The power of <mask> makes you successful")

[{'score': 0.11818496137857437,
  'token': 268,
  'token_str': ' the',
  'sequence': 'The power of the makes you successful'},
 {'score': 0.047790493816137314,
  'token': 348,
  'token_str': ' this',
  'sequence': 'The power of this makes you successful'},
 {'score': 0.04715990275144577,
  'token': 329,
  'token_str': ' which',
  'sequence': 'The power of which makes you successful'},
 {'score': 0.04489077627658844,
  'token': 263,
  'token_str': ' a',
  'sequence': 'The power of a makes you successful'},
 {'score': 0.031651124358177185,
  'token': 396,
  'token_str': ' all',
  'sequence': 'The power of all makes you successful'}]